In [1]:
from resnet import ResNetCIFAR
from train_util import *
import torch
import numpy as np
import matplotlib.pyplot as plt

import time

import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch.optim as optim

import torch.multiprocessing as mp
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

from FP_layers import *
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(device, 'torch.distributed.is_available():', torch.distributed.is_available())
print(device)
# torch.distributed.init_process_group()

cuda


In [3]:
# from lars.lars import LARS
head = nn.Sequential(FP_Linear(64, 64, Nbits=None), nn.ReLU(True), FP_Linear(64, 64, Nbits=None))

# model = nn.parallel.DistributedDataParallel(ResNetCIFAR(head_g=head))
# model = nn.DataParallel(ResNetCIFAR(head_g=head))

batch_size = int(4096)
world_size = torch.cuda.device_count()
print(world_size)
mp.spawn(train, args=(world_size, 100, batch_size, 0.3*batch_size/256, 1e-6, head, 50,), nprocs=world_size)
# rank, world_size, epochs, batch_size, lr, reg, head, log_every_n=50):
# train(model, epochs=100, batch_size=batch_size, lr=0.3*batch_size/256, reg=1e-6, world_size=world_size, log_every_n=50)
# net, epochs, batch_size, lr, reg, rank, world_size, log_every_n=50
# optimizer = LARS(model.parameters(), lr=0.1, momentum=0.9)
# optimizer.zero_grad()
# loss_fn(model(input), target).backward()
# optimizer.step()

2


ProcessRaisedException: 

-- Process 0 terminated with the following error:
Traceback (most recent call last):
  File "/home/zl310/anaconda3/envs/michael/lib/python3.8/site-packages/torch/multiprocessing/spawn.py", line 69, in _wrap
    fn(i, *args)
  File "/home/zl310/ece661-hw-fa23/project_test/train_util.py", line 75, in train
    net = DDP(ResNetCIFAR(head_g=head), device_ids=[rank])
  File "/home/zl310/anaconda3/envs/michael/lib/python3.8/site-packages/torch/nn/parallel/distributed.py", line 603, in __init__
    self._log_and_throw(
  File "/home/zl310/anaconda3/envs/michael/lib/python3.8/site-packages/torch/nn/parallel/distributed.py", line 769, in _log_and_throw
    raise err_type(err_msg)
ValueError: DistributedDataParallel device_ids and output_device arguments only work with single-device/multiple-device GPU modules or CPU modules, but got device_ids [0], output_device None, and module parameters {device(type='cpu')}.


In [ ]:
# # b = torch.rand((64, 3, 28, 28))
# head = nn.Sequential(FP_Linear(64, 64, Nbits=None), nn.ReLU(True), FP_Linear(64, 64, Nbits=None))

# model = ResNetCIFAR(head_g=head).to(device)
# trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=False, transform=transforms.ToTensor())
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=1)
# b = None
# for idx, (inputs, label) in enumerate(trainloader):
#     print(inputs.shape)
#     b = inputs.to(device)
#     break

# y = model(b)

# # xcs = F.cosine_similarity(y[None, :, :], y[:,None,:], dim=-1)
# # ce_loss = F.cross_entropy(xcs / temperature, target, reduction="mean")
# # diff_sum = 0
# # for rr, cc in enumerate(range(128)):
#     # diff_sum += xcs[rr, cc] - F.cosine_similarity(y[rr, :], y[cc, :], dim=-1)

torch.Size([128, 3, 32, 32])


tensor([1, 0, 3, 2, 5, 4, 7, 6])
